<a href="https://colab.research.google.com/github/HJJunn/Mileage-prediction/blob/main/%EC%9D%B4%EC%BB%A4%EB%A8%B8%EC%8A%A4_%EA%B3%A0%EA%B0%9D_%EC%84%B8%EB%B6%84%ED%99%94_%EC%B5%9C%EC%A2%85%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

In [ ]:
Discount = pd.read_csv("/content/drive/MyDrive/데이터분석기초/Discount_info.csv")
Customer = pd.read_csv("/content/drive/MyDrive/데이터분석기초/Customer_info.csv")
Marketing = pd.read_csv("/content/drive/MyDrive/데이터분석기초/Marketing_info.csv")
OnlineSales = pd.read_csv("/content/drive/MyDrive/데이터분석기초/Onlinesales_info.csv")
Tax = pd.read_csv("/content/drive/MyDrive/데이터분석기초/Tax_info.csv")

# (1) 데이터 확인

In [ ]:
Discount

In [ ]:
Discount.info()

In [ ]:
Discount.isnull().sum()

In [ ]:
Customer

In [ ]:
Marketing

In [ ]:
OnlineSales

In [ ]:
Tax

#(2) 데이터 전처리


In [ ]:
Marketing['날짜']= pd.to_datetime(Marketing.날짜)
Marketing.info()

In [ ]:
#OnlineSales + Customer
df1 = pd.merge(OnlineSales, Customer, how = 'left', left_on = '고객ID', right_on= '고객ID')
df1

## '월' 파생변수 생성 후 데이터 합치기

In [ ]:
df1['거래날짜'] = pd.to_datetime(df1.거래날짜)

In [ ]:
df1['월'] = df1['거래날짜'].dt.month

In [ ]:
def toMonth(x):
    month = ''
    if x == 1:
        month = 'Jan'
    elif x == 2:
        month = 'Feb'
    elif x == 3:
        month = 'Mar'
    elif x == 4:
        month = 'Apr'
    elif x == 5:
        month = 'May'
    elif x == 6:
        month = 'Jun'
    elif x == 7:
        month = 'Jul'
    elif x == 8:
        month = 'Aug'
    elif x == 9:
        month = 'Sep'
    elif x == 10:
        month = 'Oct'
    elif x == 11:
        month = 'Nov'
    elif x == 12:
        month = 'Dec'

    return month

In [ ]:
df1['월'] = df1['월'].apply(lambda x : toMonth(x))
df1.head()

In [ ]:
#df1 + Discount
df2 = pd.merge(df1, Discount, how = 'left' , left_on = ['제품카테고리', '월'], right_on = ['제품카테고리','월'])
df2

## 결측값 확인


In [ ]:
#결측값 확인
df2.isnull().sum()

In [ ]:
#쿠폰코드가 null인 데이터 400개
null_쿠폰코드 = df2[df2['쿠폰코드'].isnull()]
null_쿠폰코드

In [ ]:
#쿠폰코드와 할인율 모두 null인 데이터 6512개
#결측값 있는 곳은 쿠폰코드와 할인율이 동시에 null
df2[df2['쿠폰코드'].isnull() & df2['할인율'].isnull()]

## 할인율 결측값 대체

In [ ]:
# 월별 할인율 확인
Discount.groupby(by = ['월'])['할인율'].mean().reset_index()

In [ ]:
#결측값 평균값으로 대체
df2.loc[(df2['월'].isin(['Jan', 'Apr', 'Jul', 'Oct'])) & (df2['할인율'].isnull()), '할인율'] = 10.0
df2.loc[(df2['월'].isin(['Feb', 'May', 'Aug', 'Nov'])) & (df2['할인율'].isnull()), '할인율'] = 20.0
df2.loc[(df2['월'].isin(['Mar', 'Jun', 'Sep', 'Dec'])) & (df2['할인율'].isnull()), '할인율'] = 30.0

In [ ]:
df2.isnull().sum()

## 고객 생애 가치(CLV) 구하기


**고객 생애 가치 구하는 식**

**고객 가치 = 평균 구매 가치 x 평균 구매 빈도**

**평균 구매 가치 = 특정 기간 내 총 수익/특정 기간 내 구매 횟수**

**CLV = 고객 가치 x 평균 고객 유지 기간**

In [ ]:
#쿠폰 상태에 따라 실제수익 구하기
def total_revenue(row):
    if row['쿠폰상태'] == 'Used' :
        revenue = (row['수량'] * row['평균금액']) * (1 - row['할인율']/100) + row['배송료']
    else :
        revenue = (row['수량'] * row['평균금액'])+ row['배송료']
    return revenue

df2['실제수익'] = df2.apply(lambda x : total_revenue(x), axis = 1)

In [ ]:
##평균 구매 가치
df3 = df2.groupby(by = '고객ID')[['거래ID', '실제수익','거래날짜', '가입기간']].agg(
    구매횟수 = ('거래ID','count'),
    총수익 = ('실제수익', 'sum'),
    거래날짜 = ('거래날짜', 'max'),
    가입기간 = ('가입기간', 'mean')
)

#마지막 거래날짜 계산
df3['거래날짜'] = pd.to_datetime(df3.거래날짜)
df3['거래날짜'] = dt.datetime(2020,1,1) - df3['거래날짜']
df3['거래날짜'] = df3['거래날짜'].apply(lambda x: x.days+1)


df3['평균구매가치'] = df3['총수익']/df3['구매횟수']
df3

In [ ]:
#고객 가치
df3['평균구매빈도'] = df3['구매횟수']/12  #1년간의 평균구매빈도 구하기
df3['고객가치'] = df3['평균구매가치'] * df3['평균구매빈도']
df3

In [ ]:
df3['가입기간']= df3['가입기간']/12
df3['clv'] = df3['고객가치']*df3['가입기간']
df3


# (3) 고객 등급별 세분화

## 가중치 선정하기

In [ ]:
df3 = df3.rename(columns={'거래날짜' : 'Recency',
                         '구매횟수' : 'Frequency',
                         '총수익' : 'Monetary'})
df3

In [ ]:
# 랜덤 포레스트 모델 생성
X = df3[['Recency', 'Frequency', 'Monetary']]
y = df3['clv']
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X, y)

# 변수 중요도 확인
feature_importances = rf_model.feature_importances_
feature_names = X.columns

# 변수 중요도 출력
for i, (feature, importance) in enumerate(zip(feature_names, feature_importances), start=1):
    print(f"Feature {i}: {feature} - Importance: {importance}")

**RFM 점수 생성 최근거래날짜 가중치 : 0.1 고객별총구매수 가중치 : 0.2 고객별사용총금액 : 0.7**

In [ ]:
df_rfm_ID = df3[['Recency', 'Frequency', 'Monetary']].reset_index()
df_rfm = df_rfm_ID.drop('고객ID', axis = 1)
df_rfm

## 데이터 분포 탐색

In [ ]:
#데이터 분포 탐색

# 박스 플롯으로 4분위 수 시각화
plt.figure(figsize=(10, 6))  # 시각화 창 크기 설정

# 각 변수별로 박스 플롯 그리기
for i, column in enumerate(df_rfm.columns):
    plt.subplot(1, len(df_rfm.columns), i+1)  # 여러 개의 그래프를 나란히 그리기 위한 설정
    sns.boxplot(y=df_rfm[column])
    plt.title(column)  # 각 그래프에 칼럼 이름으로 제목 설정

plt.tight_layout()  # 그래프 간격 조정
plt.show()

In [ ]:
# 정규분포 시각화

plt.figure(figsize=(12, 8))
for i, col in enumerate(df_rfm.columns):
    plt.subplot(3, 1, i + 1)
    sns.histplot(df_rfm[col], kde=True)
    plt.title(f'{col}')

plt.tight_layout()
plt.show()

In [ ]:
#이상치의 발생과 쏠림현상을 해결하기위하여 제곱근 변환과 로그변환을 실시한다

df_rfm_log=df_rfm.copy()

df_rfm_log['Recency'] = np.sqrt(df_rfm_log['Recency'])
df_rfm_log['Frequency'] = np.log1p(df_rfm_log['Frequency'])
df_rfm_log['Monetary'] = np.log1p(df_rfm_log['Monetary'])

In [ ]:
# 박스 플롯으로 4분위 수 시각화
plt.figure(figsize=(12, 8))  # 시각화 창 크기 설정

# 각 변수별로 박스 플롯 그리기
for i, column in enumerate(df_rfm_log.columns):
    plt.subplot(2, len(df_rfm_log.columns), i+1)  # 여러 개의 그래프를 나란히 그리기 위한 설정
    sns.boxplot(y=df_rfm_log[column])
    plt.title(column)  # 각 그래프에 칼럼 이름으로 제목 설정

plt.figure(figsize=(12, 8))
for i, col in enumerate(df_rfm_log.columns):
    plt.subplot(2, 3, i + 1)
    sns.histplot(df_rfm_log[col], kde=True)
    plt.title(f'{col}')

plt.tight_layout()
plt.show()
plt.show()

## RFM 등급 나누기

In [ ]:
df_rfm_log.describe()

In [ ]:
# R,F,M 구간별 등급
df_rfm_ID['R'] = pd.qcut(df_rfm_log['Recency'], q = [0, 0.2,0.4, 0.6, 0.8, 1],
                              labels=[1,2,3,4,5]).astype(int)
df_rfm_ID['F'] = pd.qcut(df_rfm_log['Frequency'], q = [0, 0.2,0.4, 0.6, 0.8, 1],
                              labels=[1,2,3,4,5]).astype(int)
df_rfm_ID['M'] = pd.qcut(df_rfm_log['Monetary'],  q = [0, 0.2,0.4, 0.6, 0.8, 1],
                              labels=[1,2,3,4,5]).astype(int)
df_rfm_ID.head()

In [ ]:
df_rfm_ID['RFM_Score'] = df_rfm_ID['R'] * 0.1 + df_rfm_ID['F'] * 0.2 + df_rfm_ID['M'] * 0.7
df_rfm_ID

In [ ]:
# RFM 스코어를 기반으로 등급 나누기
df_rfm_ID['RFM_Rank'] = pd.cut(df_rfm_ID['RFM_Score'], bins=[-np.inf, 2, 3, 4, np.inf], labels=['BRONZE', 'SILVER', 'GOLD', 'VIP'])
df_rfm_ID

In [ ]:
df_rfm_ID['RFM_Rank'].value_counts()

In [ ]:
#등급 분포도 시각화
fig = px.scatter(
    df_rfm_ID,
    x = df_rfm_ID['고객ID'].index,
    y = 'RFM_Score',
    color = 'RFM_Rank'
)
fig.show()

# (4) 고객분석


In [ ]:
df4 = pd.merge(df2, df_rfm_ID, how = 'left', left_on = '고객ID', right_on = '고객ID')
df4

## 성별

In [ ]:
df4_gender = df4['성별'].value_counts().reset_index()
df4_gender

In [ ]:
fig = px.pie(
    df4_gender,
    names='성별',
    values = 'count',
    title='전체 성별',
    hole = 0.3
)
fig.show()

In [ ]:
#고객 등급별 성별
df_gender = df4.groupby(by = '고객ID')[['RFM_Rank', '성별']].first().reset_index()
gender_counts = df_gender.groupby(['RFM_Rank', '성별']).size().unstack()
gender_counts

In [ ]:
fig = px.bar(gender_counts,
             x=gender_counts.index,
             y=['남', '여'],
             title='등급별 성별 분포',
             labels={'x': 'RFM Rank', 'y': 'Count', '성별': '성별'},
             barmode='group')

fig.show()

## 지역

In [ ]:
#고객 등급별 지역
df_region = df4.groupby(by = '고객ID')[['RFM_Rank', '고객지역']].first().reset_index()
region_counts = df_region.groupby(['RFM_Rank', '고객지역']).size().unstack()
region_counts

In [ ]:
fig = px.bar(region_counts,
             x=region_counts.index,
             y=region_counts.columns,
             title='등급별 지역 분포',
             labels={'x': 'RFM Rank', 'y': 'Count', '고객지역': '고객지역'},
             barmode='group')

fig.show()

## 월간 거래수, 총수익

In [ ]:
df2['월'] = df2['거래날짜'].dt.month
df_제품 = df2.groupby(by = ['월','제품카테고리']).agg(
    총수익 = ('실제수익', 'sum'),
    거래수 = ('거래ID', 'count')
).reset_index()
df_제품

In [ ]:
#상위 품목 3개를 제거해서 하위 품목 확인
filtered_제품 = ['Nest-USA', 'Apparel']
filtered_df = df_제품[~df_제품['제품카테고리'].isin(filtered_제품)]
filtered_df

In [ ]:
fig1 = px.line(
    df_제품,
    x = '월',
    y= '총수익',
    color = '제품카테고리',
    title = '월간 제품별 총수익'
)

fig2 = px.line(
    filtered_df,
    x = '월',
    y= '총수익',
    color = '제품카테고리',
    title = '필터링 된 월간 제품별 총수익'
)
fig1.show()
fig2.show()

In [ ]:

fig3 = px.line(
    df_제품,
    x = '월',
    y= '거래수',
    color = '제품카테고리',
    title = '월간 제품별 거래 수'
)
fig4 = px.line(
    filtered_df,
    x = '월',
    y= '거래수',
    color = '제품카테고리',
    title = '필터링 된 월간 제품별 거래 수'
)
fig3.show()
fig4.show()

## 2. 등급별 많이 판매된 제품

In [ ]:
df4['거래날짜'] = pd.to_datetime(df4.거래날짜)
df4['월_num'] = df4['거래날짜'].dt.month

In [ ]:
rank_제품 = df4.groupby(by = ['월_num','제품카테고리','RFM_Rank']).agg(
    총수익 = ('실제수익', 'sum'),
    거래수 = ('거래ID', 'count')
).reset_index()
rank_제품

In [ ]:
vip = rank_제품[rank_제품['RFM_Rank'] == 'VIP']
vip_top = vip.sort_values(by = '거래수',ascending = False).head(10)
vip_top

In [ ]:
gold = rank_제품[rank_제품['RFM_Rank'] == 'GOLD']
gold_top = gold.sort_values(by = '거래수',ascending = False).head(10)
gold_top

In [ ]:
silver = rank_제품[rank_제품['RFM_Rank'] == 'SILVER']
silver_top = silver.sort_values(by = '거래수',ascending = False).head(10)
silver_top

In [ ]:
bronze = rank_제품[rank_제품['RFM_Rank'] == 'BRONZE']
bronze_top = bronze.sort_values(by = '거래수',ascending = False).head(10)
bronze_top

In [ ]:
fig_bronze = px.line(
    bronze,
    x = '월_num',
    y = '거래수',
    color = '제품카테고리',
    title = 'BRONZE 월별 카테고리 거래 수'
)
fig_silver = px.line(
    silver,
    x = '월_num',
    y = '거래수',
    color = '제품카테고리',
    title = 'SILVER 월별 카테고리 거래 수'
)
fig_gold = px.line(
    gold,
    x = '월_num',
    y = '거래수',
    color = '제품카테고리',
    title = 'GOLD 월별 카테고리 거래 수'
)
fig_vip = px.line(
    vip,
    x = '월_num',
    y = '거래수',
    color = '제품카테고리',
    title = 'VIP 월별 카테고리 거래 수'
)
fig_bronze.show()
fig_silver.show()
fig_gold.show()
fig_vip.show()

## 등급별 월간 거래 현황

In [ ]:
rank_총수익 = rank_제품.groupby(by='RFM_Rank')[['총수익', '거래수']].sum().reset_index()
rank_총수익

In [ ]:
#월간 거래수
df_거래 =rank_제품.groupby(by = ['월_num','RFM_Rank'])[['거래수', '총수익']].sum().reset_index()
fig = px.line(
    df_거래,
    x = '월_num',
    y = '거래수',
    color = 'RFM_Rank',
)
fig.show()

In [ ]:
#월간 총수익
fig = px.line(
    df_거래,
    x = '월_num',
    y = '총수익',
    color = 'RFM_Rank',
)
fig.show()

## 쿠폰 사용량

In [ ]:
#쿠폰사용량
df_쿠폰 =  df4[['RFM_Rank', '제품카테고리', '쿠폰상태','거래ID']]
df_쿠폰['쿠폰상태'] = df_쿠폰['쿠폰상태'].apply(lambda x: 1 if x == 'Used' else 0)
df_쿠폰 = df_쿠폰.groupby(by = ['RFM_Rank', '제품카테고리'])[['쿠폰상태', '거래ID']].agg(
    쿠폰거래=('쿠폰상태', 'sum'),
    거래수=('거래ID', 'count')
).reset_index()
df_쿠폰['쿠폰사용비율'] = df_쿠폰['쿠폰거래']/df_쿠폰['거래수']
df_쿠폰

# 쿠폰 used 한 고객들 뽑아냄
# 거래 날짜, 제품 카테고리, 쿠폰 사용 used count

In [ ]:
fig = px.bar(
    df_쿠폰,
    x = '제품카테고리',
    y= '쿠폰사용비율',
    color = 'RFM_Rank',
    hover_data=['거래수', '쿠폰거래'],
    title='제품 카테고리별 등급에 따른 쿠폰 사용 비율'
)
fig.show()

VIP등급에선 More Bags의 쿠폰 사용비율이 가장 높음

GOLD등급에선 Fun, Notebooks & Journals 쿠폰 사용비율이 높음

SILVER등급에선 표본은 적지만 Backpacks, Google, Gift Card의 쿠폰 사용비율이 높음

BRONZE등급에선 Apparel,Nest-Canada, Office 등이 높음

**=> 등급에 따라 쿠폰 사용율이 높은 제품에 쿠폰을 제공할 수 있다.**

# (5) 제품분석

In [ ]:
# dfdf = df4.groupby(by = ['거래날짜', '제품카테고리'])['거래ID'].count()
df_corr = df4.pivot_table(
    index = '거래날짜',
    columns = '제품카테고리',
    values = '거래ID',
    aggfunc = 'count'

).fillna(0)
df_corr.head()

## 제품 별 상관관계

In [ ]:
corr = df_corr.corr()
plt.figure(figsize = (10,9))
sns.heatmap(corr, annot = True, fmt = '.2f')


**'Apparel', 'Notebooks & Journals' 강한 양의 상관관계**

**'Nest', 'Accessories' 양의 상관관계**

**'Nest', 'Notebooks & Journals' 음의 상관관계**

**'Accessories', 'Notebooks & Journals' 음의 상관관계**





In [ ]:
fig = px.scatter_matrix(
    df_corr[['Accessories','Apparel', 'Notebooks & Journals','Nest']],
    height = 800,
    width = 1000,
    opacity = .5
)
fig.show()

Nest와 Accessories는 양의 상관관계로 판매량이 높은 Nest를 기준으로 Accessories 상품을 추천해 줄 수 있다. 또한 Notebooks & Journals는 Nest와 Accessories와 음의 상관관계를 띄고 있으므로 대체 상품 등으로 활용할 수 있다.

판매량이 높은 Apparel를 통해 강한 양의 상관관계를 띄는 Notebooks & Journals를 고객에게 추천함으로써 Notebooks & Journals의 판매량을 높일 수 있다.

# (5) 마케팅 분석

In [ ]:
#월별 평균 마케팅 비용 확인
Marketing['월'] = Marketing['날짜'].dt.month
df_마케팅 = Marketing.groupby(by = '월')[['오프라인비용', '온라인비용']].mean().reset_index()
df_마케팅['총 마케팅비용'] = df_마케팅['오프라인비용'] + df_마케팅['온라인비용']
df_마케팅

In [ ]:
df_melted = pd.melt(df_마케팅, id_vars='월', value_vars=['오프라인비용', '온라인비용', '총 마케팅비용'],
                    var_name='비용 유형', value_name='비용')

In [ ]:
fig = px.line(df_melted,
              x='월',
              y='비용',
              color='비용 유형',
              title='월별 평균 온라인 및 오프라인 비용')
fig.show()

In [ ]:

df_월_수익_거래 = df4.groupby(by = '월_num').agg(
    총수익 = ('실제수익', 'sum'),
    거래수 = ('거래ID', 'count')
).reset_index()
df_마케팅_거래 = pd.merge(df_마케팅,df_월_수익_거래, how = 'left', left_on = '월', right_on = '월_num')
df_마케팅_거래.drop('월_num',axis = 1)

## 회귀분석

In [ ]:
# 온, 오프라인 비용이 총 수익에 어떤 영향을 끼치는 확인하기 위해 선형회귀 진행
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

X = df_마케팅_거래[['오프라인비용', '온라인비용']]
y = df_마케팅_거래['총수익']

# 특성 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터를 훈련 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 선형 회귀 모델 생성
model = LinearRegression()

# 모델 훈련
model.fit(X_train, y_train)

# 테스트 세트를 사용하여 예측
y_pred = model.predict(X_test)

# 회귀 계수 출력
print("회귀 계수 (기울기):", model.coef_)


**오프라인 비용의 계수: 15090.43, 이는 오프라인 비용이 1 표준편차 증가할 때 총수익이 약 15090.43 단위 증가함을 의미합니다.**


**온라인 비용의 계수: 23097.36, 이는 온라인 비용이 1 표준편차 증가할 때 거래 수가 약 23097.36 단위 증가함을 의미합니다.**

## 산점도

In [ ]:
# 산점도 오프라인비용
fig = px.scatter(df_마케팅_거래, x='총 마케팅비용', y='총수익',
                 title='총 마케팅 비용에 따른 월별 거래 수와 총수익-오프라인비용',
                 color = '오프라인비용',
                 hover_name = '월',
                 size = '거래수'
)

# 그래프 표시
fig.show()
# 산점도 온라인비용
fig2 = px.scatter(df_마케팅_거래, x='총 마케팅비용', y='총수익',
                 title='총 마케팅 비용에 따른 월별 거래 수와 총수익-온라인비용',
                 color = '온라인비용',
                 hover_name = '월',
                 size = '거래수'
)

# 그래프 표시
fig2.show()

**12월의 경우 총마케팅비용이 가장 높으며 총수익도 가장 많았다.**

**11월의 경우 총마케팅비용은 12월보다 비교적 적었지만 총수익은 비슷했으며 오프라인 비용보다 온라인비용이 더 큰 것으로 온라인 마케팅 효과가 크다는 것을 알 수 있다.**

**8월의 경우 총마케팅비용과 총수익이 많지는 않지만 거래수 가장 많은 것으로 보아 이때 할인을 많이 했다는 것을 유추할 수 있다.**

**2월의 경우 총 마케팅 비용은 많지만 총수익과 거래수가 적어 마케팅 효과가 미미했다는 것을 알 수 있다.**